In [1]:
import json
import os


import rootpath
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib
# matplotlib.use('Agg') # Must be before importing matplotlib.pyplot or pylab!
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import geopandas as gpd
import pycountry as pc

from lhledge import cfgLoader
from lhledge import lhlFilters
from lhledge import loadGeographicData

In [2]:

CYCLE = 8820
DATE = 20201002
# CYCLE = 4578
# DATE = 20160302
DOWNSAMPLING = 10

In [3]:
def ecdf(data, w=[]):
    """ Compute ECDF """
    if len(w) == 0:
        w = np.ones(len(data))
    #
    #     x = np.sort(data)
    idx = np.argsort(data).values
    #
    x = np.array(data)
    x = x[idx]
    w = w[idx]
    #
    n = x.size
    #     y = np.arange(1, n + 1) / n
    y = np.cumsum(w) / sum(w)
    return (np.squeeze(x), y)

In [4]:
def ecdf2(df, field, field_aux):
    df = df.sort_values(field)
    y = np.cumsum(df.groupby(field)["router_id"].count().values)
    y = y / max(y)
    x = df.drop_duplicates(field)[field].values
    return x, y

In [5]:
# Change directory to run from the root dir of the project
path = rootpath.detect(pattern=".git")
os.chdir(path)

# load config file
cfg = cfgLoader.cfgLoader("config.yml")

In [6]:
def geneate_input_filename(date, cycle):

    filename = cfg["paths"]["ark"]["consolidated-long-haul-explorations"].format(date, cycle, DOWNSAMPLING)

    # filenames._create_dir(filename)
    return filename

In [7]:
def unwrap_lon(x):
    if x < 40:
        return  x

    return -360 + x

In [8]:
lhl_d = {}

for date, cycle in [(20201002, 8820), (20201002, 8821), (20201002, 8822),]:
    lhl_d[cycle] = pd.read_csv(geneate_input_filename(date, cycle),
                               compression="bz2")
    lhl_d[cycle]["mpls_tunnel"] = lhl_d[cycle]["mpls_tunnel"].astype(bool)

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3012: DtypeWarning: Columns (2,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
cepii = loadGeographicData.load_inter_country_distances("data/external/cepii/dist_cepii.csv", 
                                                        "data/processed/min-cc-dist/min_cc_dist.csv")
cepii.head()

,cc_src,cc_dst,dist
0,AW,AW,0.0000
1,AW,AF,13257.8100
2,AW,AO,9516.9130
3,AW,AI,983.2682
4,AW,AL,9091.7420


In [10]:
filtered_geoloc_hops = {}

for cycle in lhl_d.keys():
    filtered_geoloc_hops[cycle] = lhlFilters.filter_with_cepii(cepii, lhlFilters.find_min_rtt(lhl_d[cycle]))

In [11]:
for cycle in [8821,]:
    df = pd.DataFrame()
    for _cycle in range(cycle -1, cycle + 2):
        df = df.append(filtered_geoloc_hops[_cycle])


,near_side_cc,far_side_cc,near_node_id,far_node_id,near_side_lat,far_side_lat,near_side_lon,far_side_lon,near_node_asn,far_node_asn,mpls_tunnel,diff_rtt,dist
7,AE,SB,1363707,2375,25.06570,-9.64167,55.17128,161.42500,15412.0,15412.0,True,287.843,10813.229608
10,AE,ZA,767255,767195,25.26330,-29.66803,55.30870,22.74251,37271.0,37271.0,True,217.385,7530.202000
33,AT,BR,404518,395182,48.20849,-25.42778,16.37208,-49.27306,3356.0,3549.0,False,213.780,1820.326437
34,AT,BR,406352,395181,48.20849,-25.42778,16.37208,-49.27306,3356.0,3549.0,False,213.372,1820.326437
37,AT,DE,6179,1355913,48.20849,48.15000,16.37208,11.58330,174.0,3320.0,False,231.252,763.731400
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24270,VN,FR,21702,25409,16.00000,48.85341,106.00000,2.34880,203523.0,9009.0,False,764.569,9208.554000
24278,ZA,CN,13177,1080129,-26.20227,34.77250,28.04363,113.72660,4809.0,4809.0,False,214.256,12967.770000
24279,ZA,CN,13177,1080132,-26.20227,34.77250,28.04363,113.72660,4809.0,4809.0,False,216.821,12967.770000
24323,ZA,US,1376686,12987,-26.20227,37.75100,28.04363,-97.82200,3491.0,3491.0,False,210.631,12581.720000


In [25]:
for ns, fs in df.loc[(df["diff_rtt"] > 200) & (df["near_side_cc"] == "JP")][["near_node_id", "far_node_id"]].values[:20]:
    display(lhl_d[cycle].loc[(lhl_d[cycle]["near_node_id"] == ns) & (lhl_d[cycle]["far_node_id"] == fs)].iloc[:1])

,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt
1090079,69.174.126.169,213.200.127.50,False,ae3.cr0-tyo1.ip4.gtt.net,ae9.cr1-vie2.ip4.gtt.net,62.013,230.972,1.601646e+09,1.601646e+09,10986,1663867,3257,3257,JP,AT,35.6895,139.69171,48.20849,16.37208,168.959


,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt
684018,210.130.133.37,58.138.89.229,False,210.130.133.37,tky001bb01.iij.net,20.349,230.353,1.601651e+09,1.601651e+09,1027108,1236,2497,2497,JP,AU,35.69,139.69,-17.05,128.2,210.004


,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt


,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt
1090218,69.174.126.169,89.149.132.13,False,ae3.cr0-tyo1.ip4.gtt.net,et-5-1-0.cr2-bru2.ip4.gtt.net,2.607,199.183,1.601670e+09,1.601670e+09,10986,72369,3257,3257,JP,BE,35.6895,139.69171,50.85045,4.34878,196.576


,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt
1090229,69.174.126.169,89.149.134.49,False,ae3.cr0-tyo1.ip4.gtt.net,ae11.cr1-bru2.ip4.gtt.net,2.521,199.182,1.601661e+09,1.601661e+09,10986,826477,3257,3257,JP,BE,35.6895,139.69171,50.85045,4.34878,196.661


...

,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt
1089899,69.174.126.169,141.136.110.117,False,ae3.cr0-tyo1.ip4.gtt.net,ae5.cr3-fra2.ip4.gtt.net,2.484,202.613,1.601668e+09,1.601668e+09,10986,826502,3257,3257,JP,DE,35.6895,139.69171,50.11552,8.68417,200.129


,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt


,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt
684748,210.171.225.22,212.74.67.77,False,as8220.ix.jpix.ad.jp,212.74.67.77,35.012,234.271,1.601650e+09,1.601650e+09,5047984,1256596,8220,8220,JP,DE,35.6895,139.69171,52.52437,13.41053,199.259


,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt


,near_side_addr,far_side_addr,mpls_tunnel,near_side_rdns,far_side_rdns,near_side_rtt,far_side_rtt,near_side_timestamp,far_side_timestamp,near_node_id,far_node_id,near_node_asn,far_node_asn,near_side_cc,far_side_cc,near_side_lat,near_side_lon,far_side_lat,far_side_lon,diff_rtt
1090044,69.174.126.169,213.200.121.86,False,ae3.cr0-tyo1.ip4.gtt.net,ae7.cr2-mad5.ip4.gtt.net,2.487,211.492,1.601644e+09,1.601644e+09,10986,826400,3257,3257,JP,ES,35.6895,139.69171,40.4165,-3.70256,209.005
